In [1]:
# # Debugging ONLY! autoreloads when scripts update
# %load_ext autoreload
# %autoreload 2

import numpy as np
import pandas as pd
import geopandas as gpd

import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import contextily as ctx

import os

from src import preprocessing, config

# Obtain raw data

In [2]:
# Get 2020 Decennial Census data
decennial2020_dp_raw = preprocessing.fetch_2020_demographic_profile()

GET request status: 200


In [ ]:
np.array(decennial2020_dp_raw).shape

(2328, 646)

In [3]:
# Get 2023 ACS 5yr data
acs5yr2023_raw = preprocessing.fetch_2023_acs_5yr_select()
np.array(acs5yr2023_raw).shape

GET request status: 200


(2328, 18)

# Perform initial cleaning

In [2]:
# Clean NYC census tract shapefiles
gdf = preprocessing.clean_tracts(config.TRACTS_RAW, config.TRACTS_CLEAN)

gdf.sample(n=5)

,BOROUGH,TRACT,AREA,LAT,LONG,geometry
GEOID,,,,,,
36081033405,Queens,334.05,261206,40.673418,-73.767649,"POLYGON ((-73.77118 40.67253, -73.77029 40.672..."
36047030800,Brooklyn,308,681579,40.586551,-73.979511,"POLYGON ((-73.98742 40.58513, -73.98729 40.585..."
36061005200,Manhattan,52,168893,40.737436,-73.990455,"POLYGON ((-73.99312 40.73669, -73.99265 40.737..."
36061022400,Manhattan,224,194691,40.811122,-73.946960,"POLYGON ((-73.95022 40.81148, -73.94976 40.812..."
36061009800,Manhattan,98,177061,40.755788,-73.967774,"POLYGON ((-73.97122 40.75519, -73.97096 40.755..."


In [3]:
decennial2020_dp_clean = preprocessing.clean_2020_demographic_profile()
print(decennial2020_dp_clean.shape)
decennial2020_dp_clean.sample(n=5)

(2222, 153)


,pop,pop_under5,pop_5to9,pop_10to14,pop_15to19,pop_20to24,pop_25to29,pop_30to34,pop_35to39,pop_40to44,pop_45to49,pop_50to54,pop_55to59,pop_60to64,pop_65to69,pop_70to74,pop_75to79,pop_80to84,pop_85plus,pop_16plus,pop_18plus,pop_21plus,pop_62plus,pop_65plus,pop_male,pop_male_under5,pop_male_5to9,pop_male_10to14,pop_male_15to19,pop_male_20to24,pop_male_25to29,pop_male_30to34,pop_male_35to39,pop_male_40to44,pop_male_45to49,pop_male_50to54,pop_male_55to59,pop_male_60to64,pop_male_65to69,pop_male_70to74,pop_male_75to79,pop_male_80to84,pop_male_85plus,pop_male_16plus,pop_male_18plus,pop_male_21plus,pop_male_62plus,pop_male_65plus,pop_female,pop_female_under5,pop_female_5to9,pop_female_10to14,pop_female_15to19,pop_female_20to24,pop_female_25to29,pop_female_30to34,pop_female_35to39,pop_female_40to44,pop_female_45to49,pop_female_50to54,pop_female_55to59,pop_female_60to64,pop_female_65to69,pop_female_70to74,pop_female_75to79,pop_female_80to84,pop_female_85plus,pop_female_16plus,pop_female_18plus,pop_female_21plus,pop_female_62plus,pop_female_65plus,medianage,medianage_male,medianage_female,pop_onerace,pop_white,pop_black,pop_nativeam,pop_asian,pop_hipacific,pop_other,pop_multi,racetally,racetally_white,racetally_black,racetally_nativeam,racetally_asian,racetally_hipacific,racetally_other,pop_hispanic,pop_nonhispanic,pop_hispanic_white,pop_hispanic_black,pop_hispanic_nativeam,pop_hispanic_asian,pop_hispanic_hipacific,pop_hispanic_other,pop_hispanic_multi,pop_nonhisp_white,pop_nonhisp_black,pop_nonhisp_nativeam,pop_nonhisp_asian,pop_nonhisp_hipacific,pop_nonhisp_other,pop_nonhisp_multi,pop_household,pop_household_householder,pop_household_spouse_oppsex,pop_household_spouse_samesex,pop_household_unmpartner_oppsex,pop_household_unmpartner_samesex,pop_household_child,pop_household_child_under18,pop_household_grandchild,pop_household_grandchild_under18,pop_household_otherrelatives,pop_household_nonrelatives,pop_gpquarters,pop_gpquarters_institutionalized,pop_gpquarters_institutionalized_male,pop_gpquarters_institutionalized_female,pop_gpquarters_noninst,pop_gpquarters_noninst_male,pop_gpquarters_noninst_female,households,households_married,households_married_withchild,households_cohabit,households_cohabit_withchild,households_singlemale,households_singlemale_alone,households_singlemale_alone_65plus,households_singlemale_withchild,households_singlefemale,households_singlefemale_alone,households_singlefemale_alone_65plus,households_singlefemale_withchild,households_withunder18,households_with65plus,housing,housing_occupied,housing_vacant,housing_vacant_forrent,housing_vacant_rentedunocc,housing_vacant_forsale,housing_vacant_soldunocc,housing_vacant_seasonal,housing_vacant_other,vacancyrate_homeowner,vacancyrate_rental,housing_owneroccupied,housing_renteroccupied
GEOID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
36047019500,3684,204,143,121,86,190,333,466,430,372,300,210,199,150,164,139,87,53,37,3198,3167,3109,576,480,1809,102,77,62,49,105,169,212,228,192,149,120,95,64,53,57,44,21,10,1560,1543,1507,222,185,1875,102,66,59,37,85,164,254,202,180,151,90,104,86,111,82,43,32,27,1638,1624,1602,354,295,38.2,37.7,38.9,3265,1841,917,12,240,2,253,419,4147,2189,1064,31,332,4,527,509,3175,68,18,2,4,2,191,224,1773,899,10,236,0,62,195,3670,1808,511,38,158,24,687,466,52,36,136,256,14,0,0,0,14,10,4,1807,551,240,188,24,423,309,61,10,645,404,113,57,366,385,1919,1807,112,34,2,15,10,6,45,1.9,3.2,782,1025
36005042901,3711,241,248,290,271,265,322,309,260,275,238,256,210,180,169,87,42,25,23,2887,2767,2616,438,346,1790,136,114,136,134,154,165,145,114,118,99,124,107,94,81,34,24,5,6,1382,1329,1248,204,150,1921,105,134,154,137,111,157,164,146,157,139,132,103,86,88,53,18,20,17,1505,1438,1368,234,196,33.3,31.7,34.9,3335,403,668,86,678,1,1499,376,4151,660,824,136,708,12,1811,2160,1551,159,128,78,3,0,1472,320,244,540,8,675,1,27,56,3711,1313,405,3,91,5,1278,800,99,77,383,

In [5]:
# Remove NYC tracts that are not in the DP or are unpopulated
gdf = preprocessing.remove_lowpop_tracts()
gdf.shape

(2222, 6)

In [4]:
# Clean 2023 ACS 5-year data
acs5yr2023_clean = preprocessing.clean_2023_acs_5yr_select()

print(acs5yr2023_clean.shape)
acs5yr2023_clean.sample(n=10)

(2222, 7)


,pct_sameresidence1yr,pct_englishonlyhome,pct_commute_transit,pct_commute_walk,medianhouseholdincome,pct_incomebelowpoverty,medianhomevalue
GEOID,,,,,,,
36005042901,84.9,27.2,68.1,3.0,42775.0,37.8,835000.0
36047025400,85.4,17.7,28.3,13.6,85819.0,19.7,1062500.0
36081117100,95.7,20.2,24.0,2.7,76658.0,13.2,1213600.0
36047025800,84.3,21.5,57.2,10.2,65234.0,30.5,917700.0
36081084901,91.6,5.7,34.7,7.8,67219.0,31.9,592900.0
36061011000,63.7,61.9,33.4,26.3,144856.0,2.9,1326900.0
36005034300,93.6,44.6,25.4,5.3,89554.0,2.7,513000.0
36061024302,94.7,64.7,61.6,8.4,22973.0,42.3,NaN
36005030701,92.2,76.0,38.0,2.1,121764.0,2.8,447300.0


In [5]:
acs5yr2023_clean.isna().sum()

pct_sameresidence1yr        0
pct_englishonlyhome         0
pct_commute_transit         3
pct_commute_walk            3
medianhouseholdincome      33
pct_incomebelowpoverty      3
medianhomevalue           286
dtype: int64